In [4]:

import numpy as np
import pandas as pd
import os
import datetime
import warnings
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
import pandas_ta as ta
import pandas as pd
import yfinance as yf
import numpy as np
import math
import statistics
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import urllib.request, json
from pandas import json_normalize
import math
# The first thing we want to do is import the Pandas library and set the filepath to our data file
import pandas as pd
import pandas_ta as ta
import warnings
from pandas.tseries.offsets import DateOffset
import warnings
import sklearn

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as sp
from itertools import groupby, chain
from collections import OrderedDict

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import quandl
quandl.ApiConfig.api_key = 'soxd-469x3Zp4ib_4uzc'

df_MKPRU = quandl.get("BCHAIN/MKPRU", authtoken="soxd-469x3Zp4ib_4uzc") # price
df_CPTRA = quandl.get("BCHAIN/CPTRA", authtoken="soxd-469x3Zp4ib_4uzc")
df_CPTRA['CPTRA_sma_30'] = pd.Series.to_frame(ta.sma(df_CPTRA['Value'], length=30))
df_CPTRA['CPTRA_sma_60'] = pd.Series.to_frame(ta.sma(df_CPTRA['Value'], length=60))
df_CPTRA = df_CPTRA.dropna()

def get_data_yfinance(asset, interval='1d', start='2011-01-01'):
    date_column = 'Date'
    list_of_days = ['1d', '5d', '1wk', '1mo', '3mo']
    list_of_minutes = ['1m', '2m', '5m', '15m', '30m', '60m', '90m']
    df = yf.download(asset, start=start, threads= False, interval=interval)
    df = df.reset_index()
    if interval== '1h':
        df = df.rename(columns={'index':'Date'})
    elif interval in list_of_minutes:
        df = df.rename(columns={'Datetime':'Date'})
        #df[date_column] = df[date_column].str[:]    
    # Now that we have loaded our data into the dataframe, we can preview it using the print & .head() function
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values([date_column], ascending=[True]).reset_index(drop=True).sort_values([date_column], ascending=[False])
    df = df.rename(columns={'Date':'date','Open':'open', 'High':'high', 'Low': 'low','Close':'close', 'Volume':'tradecount'})
    return(df)

# Create a Dataframe with dates
first_date = '2009-01-02'
first_date_obj = datetime.strptime(first_date, '%Y-%m-%d')
today_obj = datetime.today()
difference = today_obj - first_date_obj
total_days = difference.days
list_of_dates = []
for i in range(total_days):
    new_day = first_date_obj + timedelta(days=i)
    new_day = new_day.strftime('%Y-%m-%d')
    list_of_dates.append(new_day)
df_dates = pd.DataFrame(list_of_dates, columns=['date'])
df_dates['date'] = pd.to_datetime(df_dates['date'], format='%Y-%m-%d')

df_close_yf = get_data_yfinance('BTC-USD')
df_close_yf = df_close_yf[['date','close']]
df_MKPRU_quandul = df_MKPRU.reset_index().rename(columns={'Date':'date', 'Value':'close'})
df_close_clean = pd.merge(df_dates,df_MKPRU_quandul, on='date', how='left')
df_close_clean = pd.merge(df_close_clean,df_close_yf, on='date', how='left')
df_close_clean['close'] = df_close_clean[['close_x','close_y']].apply(lambda x: x['close_x'] if(np.all(pd.notnull(x['close_x']))) else x['close_y'], axis=1)
df_close_clean = df_close_clean[['date','close']]

# Creation of Maximum Series

df_CPTRA = df_CPTRA.reset_index().rename(columns={'Value':'CPTRA'})
first_date = df_CPTRA.iloc[0].Date
df_CPTRA[df_CPTRA['Date']==first_date].iloc[0].CPTRA
maximum_date_CPTRA = {}
today = datetime.today()
number_of_years = today.year - first_date.year
low_date = first_date
maximum_value=0
list_of_dates=[]
list_of_values=[]
for i in range(number_of_years+1):
    low_date =  first_date + dt.timedelta(days = 365*i)
    high_date =  first_date + dt.timedelta(days = 365*(i+1))
    df_filtered = df_CPTRA[(df_CPTRA['Date'] > low_date) &  (df_CPTRA['Date'] <= high_date)]
    maximum = df_filtered[df_filtered['CPTRA_sma_30']==df_filtered['CPTRA_sma_30'].max()].reset_index().iloc[0].CPTRA
    if maximum > maximum_value:
        maximum_value = maximum
        list_of_dates.append(df_filtered[df_filtered['CPTRA_sma_30']==df_filtered['CPTRA_sma_30'].max()].reset_index().iloc[0].Date)
        list_of_values.append(maximum)
dataframe_dates_maximum = pd.DataFrame(
    {'date': list_of_dates,
     'CPTRA_sma_30': list_of_values
    })
total_days = (today - first_date).days

# https://laid-back-scientist.com/en/fill-datetime
df_ = dataframe_dates_maximum.set_index('date')
df_ = df_.asfreq(freq='1440min')
df_fill = df_.reset_index()

# https://mohammadimranhasan.com/linear-regression-of-time-series-data-with-pandas-library-in-python/
df_dates = df_close_clean[['date']]
df_new = pd.merge(df_dates, dataframe_dates_maximum, how='left', on='date')
df_fill = df_new.copy()
df_fill =df_fill.set_index('date')
y=np.array(df_fill['CPTRA_sma_30'].dropna().values, dtype=float)
x=np.array(pd.to_datetime(df_fill['CPTRA_sma_30'].dropna()).index.values, dtype=float)
x_tot = np.array(pd.to_datetime(df_fill['CPTRA_sma_30']).index.values, dtype=float)
slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
xf = np.linspace(min(x_tot),max(x_tot),len(df_fill))
xf1 = xf.copy()
xf1 = pd.to_datetime(xf1)
yf = (slope*xf)+intercept
print('r = ', r_value, '\n', 'p = ', p_value, '\n', 's = ', std_err)
f, ax = plt.subplots(1, 1)
ax.plot(xf1, yf,label='Linear fit', lw=3, color="grey")
df_fill['CPTRA_sma_30'].dropna().plot(ax=ax,marker='o', ls='', color="black")
plt.ylabel('CPTRA (sma 30)')
ax.legend();
x_for_df = xf1.to_list()
y_for_df = np.array(yf).tolist()
df_linear_regression_max_cptra = pd.DataFrame(
    {'date': x_for_df,
     'linear_regression_max_cptra': y_for_df
    })
df_linear_regression_max_cptra = df_linear_regression_max_cptra[df_linear_regression_max_cptra['linear_regression_max_cptra']>0].reset_index(drop=True)

# Creation of Minimum Series
df_CPTRA_test =  df_CPTRA.copy()
df_CPTRA_test[df_CPTRA_test['Date']==first_date].iloc[0].CPTRA
minimum_date_CPTRA_test = {}
today = datetime.today()
number_of_years = today.year - first_date.year
low_date = first_date
list_of_minimum_dates=[]
list_of_minimum_values=[]

for i in range(int(number_of_years/2+1)):
    minimum_value=10000000000000000000000000000000000000000000000000000000000
    low_date =  first_date + dt.timedelta(days = 2*365*i)
    high_date =  first_date + dt.timedelta(days = 2*365*(i+1))
    df_filtered = df_CPTRA_test[(df_CPTRA_test['Date'] > low_date) &  (df_CPTRA_test['Date'] <= high_date)]
    minimum= df_filtered[df_filtered['CPTRA_sma_30']==df_filtered['CPTRA_sma_30'].min()].reset_index().iloc[0].CPTRA
    #if minimum < minimum_value:
        #minimum_value = minimum
    list_of_minimum_dates.append(df_filtered[df_filtered['CPTRA_sma_30']==df_filtered['CPTRA_sma_30'].min()].reset_index().iloc[0].Date)
    list_of_minimum_values.append(minimum)
dataframe_dates_minimum = pd.DataFrame(
    {'date': list_of_minimum_dates,
     'CPTRA_sma_30': list_of_minimum_values
    })
dataframe_dates_minimum = dataframe_dates_minimum[:-1] # delete last data
total_days = (today - first_date).days

# https://laid-back-scientist.com/en/fill-datetime
df_ = dataframe_dates_minimum.set_index('date')
df_ = df_.asfreq(freq='1440min')
df_fill = df_.reset_index()


df_dates = df_close_clean[['date']]
df_new = pd.merge(df_dates, dataframe_dates_minimum, how='left', on='date')
df_fill = df_new.copy()
df_fill =df_fill.set_index('date')
y=np.array(df_fill['CPTRA_sma_30'].dropna().values, dtype=float)
x=np.array(pd.to_datetime(df_fill['CPTRA_sma_30'].dropna()).index.values, dtype=float)
x_tot = np.array(pd.to_datetime(df_fill['CPTRA_sma_30']).index.values, dtype=float)
slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
xf = np.linspace(min(x_tot),max(x_tot),len(df_fill))
xf1 = xf.copy()
xf1 = pd.to_datetime(xf1)
yf = (slope*xf)+intercept
print('r = ', r_value, '\n', 'p = ', p_value, '\n', 's = ', std_err)
f, ax = plt.subplots(1, 1)
ax.plot(xf1, yf,label='Linear fit', lw=3, color="grey")
df_fill['CPTRA_sma_30'].dropna().plot(ax=ax,marker='o', ls='', color="black")
plt.ylabel('CPTRA (sma 30)')
ax.legend();

x_for_df = xf1.to_list()
y_for_df = np.array(yf).tolist()
df_linear_regression_min_cptra = pd.DataFrame(
    {'date': x_for_df,
     'linear_regression_min_cptra': y_for_df
    })
#df_linear_regression_min_cptra = df_linear_regression_min_cptra[df_linear_regression_min_cptra['linear_regression_min_cptra']>0].reset_index(drop=True)

df_final = pd.merge(df_linear_regression_max_cptra, df_CPTRA, left_on='date', right_on='Date', how='left')
df_final = pd.merge(df_final,df_close_clean, on='date', how='left')
df_final = df_final.drop(columns=['Date'])
df_final = pd.merge(df_final,df_linear_regression_min_cptra, on='date', how='left')
df_final['CPTRA_sma_60'] = pd.Series.to_frame(ta.sma(df_final['CPTRA'], length=60))
df_final = df_final[df_final['CPTRA_sma_60']>-0.1]
df_final = df_final[df_final['linear_regression_min_cptra']>-0.1]
df_final['BOB_30'] = (df_final['CPTRA_sma_30'] - df_final['linear_regression_min_cptra'])  / df_final['linear_regression_max_cptra']
df_final['BOB_60'] = (df_final['CPTRA_sma_60'] - df_final['linear_regression_min_cptra'])  / df_final['linear_regression_max_cptra']
df_final
df_final.to_csv('EOB_indicator.csv', encoding='Latin', sep=';', decimal=',', index=False)


############################################

# CALCULATION LONG AND SHORT SIGNALS

df_final['ribbon_CPTRA_30_60'] = (df_final['CPTRA_sma_30'] - df_final['CPTRA_sma_60'])/df_final['CPTRA_sma_60']

# Create previous data

def creation_df_before(df_final, days, date_column='date', list_of_columns_to_delete=['target_class', 'close']):
    df_final_before = df_final.copy()
    df_final_before = df_final_before.drop(columns=list_of_columns_to_delete)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    list_columns_df_final_before = []
    for i in df_final_before.columns:
        dictionary_change_columns[i] = 'before_1_day_' + str(i)
        list_columns_df_final_before.append('before_1_day_' + str(i))
    df_final_before = df_final_before.rename(columns=dictionary_change_columns)
    return df_final_before

def incremental_calculation(df, *args, days=1, date_column= 'date', columns_to_drop=['target_class', 'close']):
    df_final_before = creation_df_before(df_final, days, list_of_columns_to_delete=[])
    df_final_before = df.copy()
    df_final_before = df_final_before.drop(columns=columns_to_drop)
    df_final_before[date_column] = pd.DatetimeIndex(df_final_before.date) + pd.DateOffset(days)
    dictionary_change_columns = {}
    if len(args) == 0:
        list_columns_df_final_before = []
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    else:
        list_columns_df_final_before = []
        for n in args:
            list_columns_df_final_before.append(n)
        df_final_before = df_final_before[list_columns_df_final_before]
        for i in df_final_before.columns:
            dictionary_change_columns[i] = 'before_' + str(days) + '_days_' + str(i)
            if i != date_column:
                list_columns_df_final_before.append('before_' + str(days) + '_days_' + str(i))
        df_final_before = df_final_before.rename(columns=dictionary_change_columns)
        df_final_before = df_final_before.rename(columns={'before_' + str(days) + '_days_' + 'date': 'date'})
    return(df_final_before, list_columns_df_final_before)

df_final
df_sma_ribbon = df_final[['date', 'ribbon_CPTRA_30_60']].copy()
df_ribbon_before_1_days,list_columns_df_sma_before_1_days = incremental_calculation(df_sma_ribbon, days=1, columns_to_drop=[])
df_final = pd.merge(df_final, df_ribbon_before_1_days, on='date', how='inner')

def sma_long_and_short_calculation(df_final, indicator, small_sma, big_sma, date_column='date', close_column='close'):
    df_calculo = df_final[[date_column, \
                           'df_'+indicator, \
                           'df_'+ indicator +'_sma_' + str(small_sma), \
                           'df_'+indicator + '_sma_' + str(big_sma), \
                           'before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma), \
                           'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma), \
                           close_column]]
    # 2  long y short signals calculation
    df_calculo["long_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                            'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)\
                                           ]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]<0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]>=0  else 0, axis=1)
   
    df_calculo["short_signal"] = df_calculo[['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma),\
                                             'ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]].apply(lambda x: 1 if x['before_1_days_ribbon_df_' + indicator + '_'+ str(small_sma) + '_' + str(big_sma)]>0 and x['ribbon_df_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]<=0  else 0, axis=1)
    # 3 calculation maximum, minimum, differences
    list_of_operations = []
    list_of_entry_price = []
    list_of_exit_price = []
    list_of_date_entry=[]
    list_of_date_exit=[]
    list_of_maximum=[]
    list_of_minimum=[]

    maximum_value=0
    minimum_value=0
    entry_long_value=0
    entry_short_value=0
   
    date_enter_short = df_calculo.iloc[0][date_column]
    date_enter_long = df_calculo.iloc[0][date_column]
   
    for i in range(len(df_calculo)):
        if df_calculo.iloc[i]['short_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            #creation new values in long list
            entry_short_value = df_calculo.iloc[i][close_column]
            date_enter_short = df_calculo.iloc[i][date_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            minimum_value = df_calculo.iloc[i][close_column]
            maximum_value = df_calculo.iloc[i][close_column]
            if entry_short_value>0:
                list_of_operations.append('long')
                list_of_entry_price.append(entry_long_value)
                list_of_exit_price.append(entry_short_value)
                list_of_date_entry.append(date_enter_long)
                list_of_date_exit.append(date_enter_short)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)

           
        elif df_calculo.iloc[i]['long_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            date_enter_long = df_calculo.iloc[i][date_column]
            entry_long_value = df_calculo.iloc[i][close_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            maximum_value = df_calculo.iloc[i][close_column]
            minimum_value = df_calculo.iloc[i][close_column]
            if entry_short_value>0:
                list_of_operations.append('short')
                list_of_entry_price.append(entry_short_value)
                list_of_exit_price.append(entry_long_value)
                list_of_date_entry.append(date_enter_short)
                list_of_date_exit.append(date_enter_long)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)

       
        if df_calculo.iloc[i][close_column] < minimum_value:
            minimum_value = df_calculo.iloc[i][close_column]
        elif df_calculo.iloc[i][close_column] > maximum_value:
            maximum_value = df_calculo.iloc[i][close_column]
    data = pd.DataFrame(columns = ['indicator', 'operation', 'entry_price', 'exit_price', 'maximum_price', 'minimum_price', 'entry_date' ,'exit_date'])
    df_out = data.append(pd.DataFrame({'indicator':indicator, 'operation':list_of_operations, 'entry_price':list_of_entry_price, 'exit_price':list_of_exit_price, 'maximum_price':list_of_maximum, 'minimum_price':list_of_minimum, 'entry_date':list_of_date_entry ,'exit_date':list_of_date_exit}), ignore_index = True)

    return(df_out)

def sma_long_and_short_calculation(df_final, indicator, small_sma, big_sma, date_column='date', close_column='close'):
    df_calculo = df_final[[date_column, \
                           indicator, \
                           indicator +'_sma_' + str(small_sma), \
                           indicator + '_sma_' + str(big_sma), \
                           'before_1_days_ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma), \
                           'ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma), \
                           close_column,
                           'BOB_30',
                           'BOB_60']]
   
    # 2  long y short signals calculation
    df_calculo["long_signal"] = df_calculo[['before_1_days_ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma),\
                                            'ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)\
                                           ]].apply(lambda x: 1 if x['before_1_days_ribbon_' + indicator + '_'  + str(small_sma) + '_' + str(big_sma)]<0 and x['ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]>=0  else 0, axis=1)
   
    df_calculo["short_signal"] = df_calculo[['before_1_days_ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma),\
                                             'ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]].apply(lambda x: 1 if x['before_1_days_ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]>0 and x['ribbon_' + indicator + '_' + str(small_sma) + '_' + str(big_sma)]<=0  else 0, axis=1)
    df_calculo
    # 3 calculation maximum, minimum, differences
    list_of_operations = []
    list_of_entry_price = []
    list_of_exit_price = []
    list_of_date_entry=[]
    list_of_date_exit=[]
    list_of_maximum=[]
    list_of_minimum=[]
    list_of_BOB_30=[]
    list_of_BOB_60=[]
   
    BOB_30=0
    BOB_60=0
    maximum_value=0
    minimum_value=0
    entry_long_value=0
    entry_short_value=0
   
    date_enter_short = df_calculo.iloc[0][date_column]
    date_enter_long = df_calculo.iloc[0][date_column]
   
    for i in range(len(df_calculo)):
        if df_calculo.iloc[i]['short_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            #creation new values in long list
            entry_short_value = df_calculo.iloc[i][close_column]
            date_enter_short = df_calculo.iloc[i][date_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            minimum_value = df_calculo.iloc[i][close_column]
            maximum_value = df_calculo.iloc[i][close_column]
            previous_BOB_30 = BOB_30
            previous_BOB_60 = BOB_60
            BOB_30 = df_calculo.iloc[i]['BOB_30']
            BOB_60 = df_calculo.iloc[i]['BOB_60']
            if entry_short_value>0:
                list_of_operations.append('long')
                list_of_entry_price.append(entry_long_value)
                list_of_exit_price.append(entry_short_value)
                list_of_date_entry.append(date_enter_long)
                list_of_date_exit.append(date_enter_short)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)
                list_of_BOB_30.append(previous_BOB_30)
                list_of_BOB_60.append(previous_BOB_60)

           
        elif df_calculo.iloc[i]['long_signal']== 1:
            if df_calculo.iloc[i][close_column] < minimum_value:
                minimum_value = df_calculo.iloc[i][close_column]
            elif df_calculo.iloc[i][close_column] > maximum_value:
                maximum_value = df_calculo.iloc[i][close_column]
            date_enter_long = df_calculo.iloc[i][date_column]
            entry_long_value = df_calculo.iloc[i][close_column]
            previous_maximum = maximum_value
            previous_minimum = minimum_value
            maximum_value = df_calculo.iloc[i][close_column]
            minimum_value = df_calculo.iloc[i][close_column]
            previous_BOB_30 = BOB_30
            previous_BOB_60 = BOB_60
            BOB_30 = df_calculo.iloc[i]['BOB_30']
            BOB_60 = df_calculo.iloc[i]['BOB_60']
            if entry_short_value>0:
                list_of_operations.append('short')
                list_of_entry_price.append(entry_short_value)
                list_of_exit_price.append(entry_long_value)
                list_of_date_entry.append(date_enter_short)
                list_of_date_exit.append(date_enter_long)
                list_of_maximum.append(previous_maximum)
                list_of_minimum.append(previous_minimum)
                list_of_BOB_30.append(previous_BOB_30)
                list_of_BOB_60.append(previous_BOB_60)

       
        if df_calculo.iloc[i][close_column] < minimum_value:
            minimum_value = df_calculo.iloc[i][close_column]
        elif df_calculo.iloc[i][close_column] > maximum_value:
            maximum_value = df_calculo.iloc[i][close_column]
    data = pd.DataFrame(columns = ['indicator', 'operation', 'entry_price', 'exit_price', 'maximum_price', 'minimum_price', 'entry_date' ,'exit_date', 'BOB_30', 'BOB_60'])
    df_out = data.append(pd.DataFrame({'indicator':indicator, 'operation':list_of_operations, 'entry_price':list_of_entry_price, 'exit_price':list_of_exit_price, 'maximum_price':list_of_maximum, 'minimum_price':list_of_minimum, 'entry_date':list_of_date_entry ,'exit_date':list_of_date_exit, 'BOB_30': list_of_BOB_30, 'BOB_60': list_of_BOB_60}), ignore_index = True)

    return(df_out)
df_out = sma_long_and_short_calculation(df_final, 'CPTRA', 30, 60)
df_out.to_csv('long_short_signals_CPTRA_adjusted.csv', encoding='Latin', sep=';', decimal=',', index=False)


############################################

df_out

[*********************100%***********************]  1 of 1 completed


IndexError: single positional indexer is out-of-bounds